<a href="https://colab.research.google.com/github/monarkparekh/MAE-598__Design-Optimization/blob/Assignment-4/MAE_598__Design_Optimization_HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<b><h1>MAE 598 Design Optimization: 
Assignment 4</h1></b>

<h2>Name: Monark Parekh <br>
ASU ID: 1222179426</h2>


<h3><b>Question 4</b></h3>

In [424]:
import numpy as np
import torch
from torch.autograd import Variable
import matplotlib.pyplot as plt
from numpy.linalg import inv

In [425]:
X = Variable(torch.tensor([0.1000, 2.0217, 2.1217]), requires_grad=True)
Xc1 = Variable(torch.tensor([0.1000, 2.0217, 2.1217]), requires_grad=True)
Xc2 = Variable(torch.tensor([0.1000, 2.0217, 2.1217]), requires_grad=True)

In [405]:
#X = Variable(torch.tensor([1., 1., 1.]), requires_grad=True)
#Xc1 = Variable(torch.tensor([1., 1., 1.]), requires_grad=True)
#Xc2 = Variable(torch.tensor([1., 1., 1.]), requires_grad=True)

n = 3
<br>m = 2 
<br>dof = 1
<br>d = x1
<br>s = [x2,x3]

In [426]:
def objective_function(X):
  obj = (X[0]**2 + X[1]**2 + X[2]**2)
  #obj_sum =  obj.sum()
  return obj

def constraints_1(Xc1):
  c1 = (Xc1[0]**2)/4 + (Xc1[1]**2)/5 + (Xc1[2]**2)/25 - 1
  return c1
def constraints_2(Xc2):
  c2 = Xc2[0] + Xc2[1] - Xc2[2]
  return c2

In [427]:
obj = objective_function(X)
cont1 = constraints_1(Xc1)
cont2 = constraints_2(Xc2)
obj.backward()
cont1.backward()
cont2.backward()

In [428]:
def reduced_gradient():  
  gradient = X.grad.numpy()
  gradient_c1 = Xc1.grad.numpy()
  gradient_c2 = Xc2.grad.numpy()

  df_dd = gradient[0]
  df_ds = np.array([gradient[1],gradient[2]])
  dh_ds = np.matrix([[gradient_c1[1],gradient_c1[2]],[gradient_c2[1],gradient_c2[2]]])
  dh_dd = np.array([gradient_c1[0],gradient_c2[0]])

  temp  = np.matmul(inv(dh_ds),dh_dd.transpose())
  temp2 = np.matmul(df_ds,temp.transpose())
  reduced_grad = df_dd - temp2
  return [reduced_grad,df_dd,df_ds,dh_ds,dh_dd]

In [429]:
def line_search(X):
  counter = 0
  alpha = 1.
  all_alpha =[1]
  b = 0.5
  t = 0.5
  def check(alpha):
    [reduced_grad,df_dd,df_ds,dh_ds,dh_dd] = reduced_gradient()
    X_c = X.clone().detach()
    X_c = X_c.numpy()
  
    d_new = X_c[0] - alpha*reduced_grad
    temp  = np.matmul(inv(dh_ds),dh_dd.transpose())
    temp2 = reduced_grad*temp
    s_new = [X_c[1],X_c[2]] + alpha*(temp2)
    X_alpha = [d_new,s_new[0,0],s_new[0,1]]
    f_alpha = objective_function(X_alpha)
    #phi_alpha = objective_function(X_c) - alpha*t*(np.matmul(df_dd,df_dd.transpose()))
    phi_alpha = objective_function(X_c) - alpha*t*(reduced_grad**2)
    return [f_alpha,phi_alpha]

  while check(alpha)[0] > check(alpha)[1] and counter < 25:
    counter += 1
    alpha = b*alpha 
    all_alpha.append(alpha)
  
  print(all_alpha)
  return alpha


In [430]:
line_search(X)

[1, 0.5, 0.25]


0.25

In [ ]:
max_iter = 10
epsilon = 1e-3
for i in range(0,max_iter):
  Xc1 = X
  Xc2 = X
  obj = objective_function(X)
  cont1 = constraints_1(Xc1)
  cont2 = constraints_2(Xc2)
  obj.backward()
  cont1.backward()
  cont2.backward()
  if reduced_gradient()[0] > epsilon:
    alpha = line_search(X)
    